In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

In [ ]:
# silence warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [ ]:
%matplotlib inline

%reload_ext autoreload
%autoreload 2

import keras

from keras_retinanet.models import retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import json

In [ ]:
# train the model
! ../../../keras-retinanet/keras_retinanet/bin/train.py --weights ../../../resnet50_coco_best_v2.1.0.h5 \
--batch-size 2 --steps 100 --epochs 20 \
--snapshot-path ../../Data/harbor-seals/snapshots-no-other \
--random-transform \
--config ../../Data/harbor-seals/config.ini \
csv ../../Data/harbor-seals/train/annotations_final_no.csv ../../Data/harbor-seals/classes-no-other.csv \
--val-annotations ../../Data/harbor-seals/valid/annotations_final_no.csv

In [ ]:
# convert the model weights to a usable format
! ../../../keras-retinanet/keras_retinanet/bin/convert_model.py --config ../../Data/harbor-seals/config.ini  ../../Data/harbor-seals/snapshots-no-other/resnet50_csv_10.h5 ../../Data/harbor-seals/models/harbor_seal_model_no_other.h5
# run the model on test set
! ../../../keras-retinanet/keras_retinanet/bin/evaluate.py csv image-level-split/test/annotations_final_no.csv ../../Data/harbor-seals/classes-no-other.csv ../../Data/harbor-seals/models/harbor_seal_model_no_other.h5


In [ ]:
def get_session():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.compat.v1.Session(config=config)

tf.compat.v1.keras.backend.set_session(get_session())

In [ ]:
model_path = '../../Data/harbor-seals/models/harbor_seal_model_no_other.h5'

model = models.load_model(model_path, backbone_name='resnet50')

labels_to_names = {0: 'adult'}

In [ ]:
image_dir = "../../Data/harbor-seals/test/"

image_list = []
for root, dirs, files in os.walk(image_dir):
    for filename in files:
        if filename.lower().endswith(('.jpg')) and "checkpoint" not in filename:
            image_list.append(image_dir + filename)
print(len(image_list))

In [ ]:
# run the model on the test set
min_score = 0.5

detections = {}

total_time = 0

for image_path in image_list:
    
    image = read_image_bgr(image_path)
          
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start
    
    boxes /= scale  
        
    if any(score >= min_score for score in scores[0]):
        detections[image_path] = []
        
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)    
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        
            
print("Finished, time per image:", total_time/len(image_list))

In [ ]:
# output model detections
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)
        
with open('image-level-split/test_detections.json', 'w') as fp:
    json.dump(detections, fp, cls=MyEncoder)

In [ ]:
# visualize adjusted output

visualize = True

for image_path in image_list: 
    
    image = read_image_bgr(image_path)
    print(image_path)
    
    if visualize:
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
        
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start 
    
    boxes /= scale
    if any(score >= min_score for score in scores [0]):
        detections[image_path] = []
    
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        
        # don't visualize the annotations along the edge of the tile
        if b[0] == 0 or b[1] == 0 or b[2] == 499 or b[3] == 499:
            continue
        
        if visualize: 
            color = label_color(label)
            
            draw_box(draw, b, color=color)
            
            # # uncomment the following to see labels and confidence scores
            # caption = "{} {:.3f}".format(labels_to_names[label], score)
            # draw_caption(draw, b, caption)
            
    if any(score >= min_score for score in scores[0]):
        if visualize:
            plt.figure(figsize=(15, 15))
            plt.axis('off')
            plt.imshow(draw)
            plt.show()
            
print("Finished, time per image:", total_time/len(image_list))